##### Experiment 1
* Date: 2024.04.19
* instance：
* * N = 5  # Number of resources
* * T = 300  # Number of time steps
* * runs = 1000  # Number of runs to average over
* * inventory_levels = [1, 5, 10, 15, 20]  # Inventory levels for each resource type
* * E = {t: np.random.choice(N, size=np.random.randint(1, N+1), replace=False).tolist() for t in range(T)}  # Randomly generated edges


In [25]:
import numpy as np
import pandas as pd
import time
from tabulate import tabulate
from algorithm import RankBasedAlgorithm, InventoryBalancingAlgorithm, GreedyAlgorithm

# Define the experiment class
class OnlineMatchingTest:
    # Initial setup includes defining the environment and parameters like rewards
    def __init__(self, N, T, E, algorithm_class, runs, inventory_levels, rewards=None, usage_duration_distributions=None):
        # Initialization with dynamic creation of usage durations if not provided
        self.N = N
        self.T = T
        self.E = E
        self.algorithm_class = algorithm_class
        self.runs = runs
        self.rewards = rewards if rewards else {n: np.random.uniform(1, 10) for n in range(N)}
        self.usage_duration_distributions = usage_duration_distributions if usage_duration_distributions else {n: np.random.choice(range(3, 6), size=2, replace=False).tolist() for n in range(N)}
        self.inventory_levels = inventory_levels

    # Run the algorithm across multiple runs and compute statistics
    def run_algorithm(self):
        results = []
        for _ in range(self.runs):
            model = self.algorithm_class(self.N, self.T, self.E, self.rewards, self.usage_duration_distributions, self.inventory_levels)
            start_time = time.time()
            total_reward = 0
            for t in range(self.T):
                result = model.allocate_resource(t)
                if result:
                    resource, _ = result
                    if resource is not None:
                        total_reward += self.rewards[resource]
            elapsed_time = time.time() - start_time
            results.append((total_reward, elapsed_time))
        return results

    def compute_statistics(self):
        results = self.run_algorithm()
        total_rewards = [result[0] for result in results]
        elapsed_times = [result[1] for result in results]

        data = {
            'Algorithm': self.algorithm_class.__name__,
            'Mean Reward': [np.mean(total_rewards)],
            'Standard Deviation Reward': [np.std(total_rewards)],
        }
        return pd.DataFrame(data)

# Experiment parameters
N = 5  # Number of resources
T = 300  # Number of time steps
runs = 1000  # Number of runs to average over
inventory_levels = [1, 5, 10, 15, 20]  # Inventory levels for each resource type
E = {t: np.random.choice(N, size=np.random.randint(1, N+1), replace=False).tolist() for t in range(T)}  # Randomly generated edges

# Algorithms to test
# algorithms = [RankBasedAlgorithm, InventoryBalancingAlgorithm, GreedyAlgorithm]
algorithms = [RankBasedAlgorithm, InventoryBalancingAlgorithm, GreedyAlgorithm]
# Run the experiment for each algorithm
results_frames = []
for algorithm_class in algorithms:
    test_env = OnlineMatchingTest(N, T, E, algorithm_class, runs, inventory_levels)
    df = test_env.compute_statistics()
    results_frames.append(df)

# Combine and display the results
final_results = pd.concat(results_frames, ignore_index=True)
print(tabulate(final_results, headers='keys', tablefmt='psql', showindex=False))


+-----------------------------+---------------+-----------------------------+
| Algorithm                   |   Mean Reward |   Standard Deviation Reward |
|-----------------------------+---------------+-----------------------------|
| RankBasedAlgorithm          |       908.991 |                34.9473      |
| InventoryBalancingAlgorithm |       195.934 |                 2.84217e-14 |
| GreedyAlgorithm             |      1230.65  |                 0.0371872   |
+-----------------------------+---------------+-----------------------------+


##### Benchmark

In [23]:
import cvxpy as cp
import numpy as np
import pandas as pd

# Parameters
n = 5  # Number of products
T = 300  # Number of periods
rewards = np.linspace(15, 30, n)  # Prices of products
inventory_levels = [1, 5, 10, 15, 20]  # Inventory levels
probabilities = np.array([1 / (20 - i) for i in range(0,7)])  # Geometric distribution parameters
survival_probabilities = np.array([(1 - probabilities[i]) ** np.arange(T) for i in range(n)])  # Survival probabilities

# To store results for statistical analysis
results = {level: [] for level in inventory_levels}

# Run the simulation 100 times
runs = 100
for _ in range(runs):
    for inventory_level in inventory_levels:
        X = cp.Variable((T, n), nonneg=True)
        revenue = cp.sum(cp.multiply(X, rewards.reshape(1, n)))
        constraints = [
            cp.sum(X, axis=1) <= 1,  # Constraint 1
            X <= 1  # Constraint 3: X[t, i] must be between 0 and 1
        ]

        # Constraint 2: Capacity constraints for each product
        for i in range(n):
            constraints.append(cp.sum(cp.multiply(X[:, i], survival_probabilities[i])) <= inventory_level)

        # Assuming all items are always available
        l_ti = np.ones((T, n))
        constraints.append(X <= l_ti)  # Constraint 4

        # Define and solve the optimization problem
        prob = cp.Problem(cp.Maximize(revenue), constraints)
        prob.solve()

        # Store the maximum revenue result
        results[inventory_level].append(prob.value)

# Calculating mean and standard deviation of the results
summary = pd.DataFrame({
    level: {
        'Mean Revenue': np.mean(results[level]),
        'STD Revenue': np.std(results[level])
    }
    for level in inventory_levels
}).T

print(summary)

    Mean Revenue   STD Revenue
1    8208.603484  0.000000e+00
5    8898.549277  1.818989e-12
10   8972.665876  0.000000e+00
15   8996.250008  1.818989e-12
20   9000.000002  1.818989e-12


In [26]:
import cvxpy as cp
import numpy as np
import pandas as pd

# Parameters
n = 5  # Number of products
T = 300  # Number of periods
rewards = np.linspace(15, 30, n)  # Prices of products
inventory_levels = [1, 5, 10, 15, 20]  # Inventory levels
probabilities = np.array([1 / (20 - i) for i in range(5)])  # Geometric distribution parameters
survival_probabilities = np.array([(1 - probabilities[i]) ** np.arange(T) for i in range(n)])  # Survival probabilities

# To store results for statistical analysis
results = {level: [] for level in inventory_levels}
total_rewards_per_run = []  # List to store total revenue per run

# Run the simulation 100 times
runs = 1000
for _ in range(runs):
    total_revenue_per_run = 0  # Initialize total revenue for this run
    for inventory_level in inventory_levels:
        X = cp.Variable((T, n), nonneg=True)
        revenue = cp.sum(cp.multiply(X, rewards.reshape(1, n)))
        constraints = [
            cp.sum(X, axis=1) <= 1,  # Constraint 1: Total allocation in each period must not exceed 1
            X <= 1,  # Constraint 3: Each allocation must be between 0 and 1
            X <= np.ones((T, n))  # Constraint 4: Assuming all items are always available
        ]

        # Constraint 2: Capacity constraints for each product based on survival probabilities
        for i in range(n):
            constraints.append(cp.sum(cp.multiply(X[:, i], survival_probabilities[i])) <= inventory_level)

        # Define and solve the optimization problem
        prob = cp.Problem(cp.Maximize(revenue), constraints)
        prob.solve()

        # Accumulate revenue for this run
        total_revenue_per_run += prob.value

    # Store the total revenue for this run
    total_rewards_per_run.append(total_revenue_per_run)

# Calculate and print the mean and standard deviation of the total revenue across all runs
mean_total_revenue = np.mean(total_rewards_per_run)
std_total_revenue = np.std(total_rewards_per_run)

print(f"Mean Total Revenue: {mean_total_revenue}")
print(f"Standard Deviation of Total Revenue: {std_total_revenue}")

Mean Total Revenue: 44076.068646576954
Standard Deviation of Total Revenue: 0.0
